In [3]:
import os
import json
import re
from dotenv import load_dotenv
from haystack import Pipeline, component
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.builders import ChatPromptBuilder, PromptBuilder
from haystack.dataclasses import ChatMessage
from haystack.tools.tool import Tool
from haystack_integrations.document_stores.mongodb_atlas import MongoDBAtlasDocumentStore
from haystack_integrations.components.retrievers.mongodb_atlas import MongoDBAtlasEmbeddingRetriever
from haystack_experimental.chat_message_stores.in_memory import InMemoryChatMessageStore
from haystack_experimental.components.retrievers import ChatMessageRetriever
from haystack_experimental.components.writers import ChatMessageWriter
from pymongo import MongoClient
from groq import Groq
from functools import partial
from typing import List, Annotated
import warnings

warnings.filterwarnings("ignore", category=UserWarning)

In [4]:
load_dotenv()


True

## ChatGenerator


In [ ]:
@component
class GroqChatGenerator:
    def __init__(self, model: str = "llama-3.3-70b-versatile", api_key: str = None): # type: ignore
        self.client = Groq(api_key=api_key or os.environ.get("GROQ_API_KEY"))
        self.model = model
    
    @component.output_types(replies=List[ChatMessage])
    def run(self, messages: List[ChatMessage]):
        if not messages:
            raise ValueError("The 'messages' list received by GroqChatGenerator is empty.")

        groq_messages = []
        for msg in messages:
            content = ""
            if hasattr(msg, 'content'):
                content = msg.content
            elif hasattr(msg, 'text'):
                content = msg.text

            if content and hasattr(msg, 'role'):
                role = msg.role.value if hasattr(msg.role, 'value') else str(msg.role)
                groq_messages.append({"role": role.lower(), "content": content})

        if not groq_messages:
            raise ValueError(
                "The 'groq_messages' list is empty after conversion. "
                "The ChatMessage objects seem to be malformed (missing .role or .content/.text)."
            )

        response = self.client.chat.completions.create(
            model=self.model,
            messages=groq_messages,
            temperature=0.7,
            max_tokens=1000
        )
        
        return {
            "replies": [
                ChatMessage.from_assistant(response.choices[0].message.content)
            ]
        }

@component
class GroqGenerator:
    def __init__(self, model: str = "llama-3.3-70b-versatile", api_key: str = None): # type: ignore
        self.client = Groq(api_key=api_key or os.environ.get("GROQ_API_KEY"))
        self.model = model
    
    @component.output_types(replies=List[str])
    def run(self, prompt: str):
        if not prompt:
            raise ValueError("The 'prompt' received by GroqGenerator is empty.")
            
        response = self.client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.3,
            max_tokens=500
        )
        
        return {
            "replies": [response.choices[0].message.content]
        }
    
    

## Setup Components


In [6]:
# Document stores
products_document_store = MongoDBAtlasDocumentStore(
    database_name="smartshopper_store",
    collection_name="products",
    vector_search_index="vector_index",
    full_text_search_index="search_index",
)

common_info_document_store = MongoDBAtlasDocumentStore(
    database_name="smartshopper_store",
    collection_name="common_info",
    vector_search_index="common_info_vector_index",
    full_text_search_index="common_info_search_index",
)



In [7]:
# Chat message store
chat_message_store = InMemoryChatMessageStore()
chat_message_writer = ChatMessageWriter(chat_message_store)


print(f"Products in store: {products_document_store.count_documents()}")
print(f"Common info in store: {common_info_document_store.count_documents()}")

Products in store: 5
Common info in store: 6


## MongoDB Components

In [8]:
class MongoDBAtlas:
    def __init__(self, mongo_connection_string: str):
        self.client = MongoClient(mongo_connection_string)
        self.db = self.client.smartshopper_store
        self.material_collection = self.db.materials
        self.category_collection = self.db.categories

    def get_materials(self):
        return [doc['name'] for doc in self.material_collection.find()]

    def get_categories(self):
        return [doc['name'] for doc in self.category_collection.find()]

@component
class GetMaterials:
    def __init__(self):
        self.db = MongoDBAtlas(os.environ['MONGO_CONNECTION_STRING'])
    
    @component.output_types(materials=List[str])
    def run(self):
        materials = self.db.get_materials()
        return {"materials": materials}

@component  
class GetCategories:
    def __init__(self):
        self.db = MongoDBAtlas(os.environ['MONGO_CONNECTION_STRING'])
    
    @component.output_types(categories=List[str])
    def run(self):
        categories = self.db.get_categories()
        return {"categories": categories}

## All Pipeline Classes


In [ ]:
class ParaphraserPipeline:
    def __init__(self, chat_message_store):
        self.memory_retriever = ChatMessageRetriever(chat_message_store)
        self.pipeline = Pipeline()
        self.pipeline.add_component("prompt_builder", ChatPromptBuilder(
            variables=["query", "memories"],
            required_variables=["query", "memories"],
        ))
        self.pipeline.add_component("generator", GroqChatGenerator())
        self.pipeline.add_component("memory_retriever", self.memory_retriever)

        self.pipeline.connect("prompt_builder.prompt", "generator.messages")
        self.pipeline.connect("memory_retriever", "prompt_builder.memories")
    
    def run(self, query):
        messages = [
            ChatMessage.from_system("You are a helpful assistant that paraphrases user queries based on previous conversations."),
            ChatMessage.from_user("""
                Please paraphrase the following query based on the conversation history. 
                If the conversation history is empty, please return the query as is.
                
                History:
                {% for memory in memories %}
                    {{memory.text}}
                {% endfor %}
                
                Query: {{query}}
                Answer:
                """)
        ]

        res = self.pipeline.run(
            data={"prompt_builder": {"query": query, "template": messages}},
            include_outputs_from=["generator"] # type: ignore
        )
        return res["generator"]["replies"][0].text

In [10]:
class MetaDataFilterPipeline:
    def __init__(self, template):
        self.template = template
        self.pipeline = Pipeline()
        self.pipeline.add_component("materials", GetMaterials())
        self.pipeline.add_component("categories", GetCategories())
        self.pipeline.add_component("prompt_builder", PromptBuilder(
            template=self.template,
            required_variables=["input", "materials", "categories"],
        ))
        self.pipeline.add_component("generator", GroqGenerator())
        self.pipeline.connect("materials.materials", "prompt_builder.materials")
        self.pipeline.connect("categories.categories", "prompt_builder.categories")
        self.pipeline.connect("prompt_builder", "generator")

    def run(self, query: str):
        res = self.pipeline.run({"prompt_builder": {"input": query}})
        return res["generator"]["replies"][0]


In [ ]:
class RetrieveAndGenerateAnswerPipeline:
    def __init__(self, chat_message_store, document_store):
        self.chat_message_store = chat_message_store
        self.document_store = document_store
        self.pipeline = Pipeline()
        self.pipeline.add_component("embedder", SentenceTransformersTextEmbedder())
        self.pipeline.add_component("retriever", MongoDBAtlasEmbeddingRetriever(document_store=document_store, top_k=5))
        self.pipeline.add_component("prompt_builder", ChatPromptBuilder(
            variables=["query", "documents"],
            required_variables=["query", "documents"]
        ))
        self.pipeline.add_component("generator", GroqChatGenerator())
        
        self.pipeline.connect("embedder", "retriever")
        self.pipeline.connect("retriever", "prompt_builder.documents")
        self.pipeline.connect("prompt_builder.prompt", "generator.messages")

    def run(self, query: str, filter: dict = {}):
        messages = [
            ChatMessage.from_system("You are a helpful shop assistant that provides product recommendations."),
            ChatMessage.from_user("""
                Generate a list of products that best match the query.

                **Query Summary:** {{query}}

                **Recommended Products:**
                {% if documents|length > 0 %}
                {% for product in documents %}
                {{loop.index}}. **{{ product.meta.title }}**
                   - Price: Rp {{ "{:,}".format(product.meta.price) }}
                   - Material: {{ product.meta.material }}
                   - Category: {{ product.meta.category }}
                   - Brand: {{ product.meta.brand }}
                   - Why recommended: [Based on product description and user needs]

                {% endfor %}
                {% else %}
                No matching products found for your query.
                {% endif %}

                Answer:
                """)
        ]
        
        res = self.pipeline.run({
            "embedder": {"text": query},
            "retriever": {"filters": filter},
            "prompt_builder": {"query": query, "template": messages}
        }, include_outputs_from=["generator"]) # type: ignore
        
        return res["generator"]["replies"][0].text

In [ ]:
class CommonInfoPipeline:
    def __init__(self, document_store):
        self.document_store = document_store
        self.pipeline = Pipeline()
        self.pipeline.add_component("embedder", SentenceTransformersTextEmbedder())
        self.pipeline.add_component("retriever", MongoDBAtlasEmbeddingRetriever(document_store=document_store, top_k=3))
        self.pipeline.add_component("prompt_builder", ChatPromptBuilder(
            variables=["query", "documents"],
            required_variables=["query", "documents"]
        ))
        self.pipeline.add_component("generator", GroqChatGenerator())
        
        self.pipeline.connect("embedder", "retriever")
        self.pipeline.connect("retriever", "prompt_builder.documents")
        self.pipeline.connect("prompt_builder.prompt", "generator.messages")

    def run(self, query: str):
        messages = [
            ChatMessage.from_system("You are a helpful customer service assistant."),
            ChatMessage.from_user("""
                Based on the retrieved information, provide a helpful answer to the user's question.

                Retrieved Information:
                {% for doc in documents %}
                **{{ doc.meta.title }}:**
                {{ doc.content }}
                ---
                {% endfor %}

                User Question: {{query}}

                Please provide a comprehensive and friendly answer:
                """)
        ]
        
        res = self.pipeline.run({
            "embedder": {"text": query},
            "prompt_builder": {"query": query, "template": messages}
        }, include_outputs_from=["generator"]) # type: ignore
        
        return res["generator"]["replies"][0].text

## Template


In [13]:
METADATA_FILTER_TEMPLATE = """
Based on the user input, create a filter for products using Haystack filter syntax.

Available materials: {{materials}}
Available categories: {{categories}}

IMPORTANT: Use Haystack filter syntax format. Return ONLY valid JSON without markdown formatting.

Examples of valid filter syntax:
- For category: {"field": "category", "operator": "in", "value": ["Electronics"]}
- For material: {"field": "material", "operator": "in", "value": ["Cotton"]}
- For price range: {"field": "price", "operator": ">=", "value": 100000}
- Multiple filters: {"operator": "AND", "conditions": [{"field": "category", "operator": "in", "value": ["Electronics"]}, {"field": "price", "operator": "<=", "value": 5000000}]}

User input: {{input}}

Return only the filter JSON without any markdown or explanation:
"""

## Initialize All Components


In [14]:
print("Initializing all components...")

paraphraser = ParaphraserPipeline(chat_message_store)
metadata_filter = MetaDataFilterPipeline(METADATA_FILTER_TEMPLATE)
rag_pipeline = RetrieveAndGenerateAnswerPipeline(chat_message_store, products_document_store)
common_info_pipeline = CommonInfoPipeline(common_info_document_store)

Initializing all components...


## Tool Functions


In [15]:
def normalize_filter_format(filter_dict):
    """Convert simple filter format to Haystack filter format"""
    if not filter_dict:
        return {}
    
    conditions = []
    
    # Handle category filter
    if "category" in filter_dict and filter_dict["category"]:
        conditions.append({
            "field": "category",
            "operator": "in", 
            "value": filter_dict["category"]
        })
    
    # Handle material filter
    if "material" in filter_dict and filter_dict["material"]:
        conditions.append({
            "field": "material",
            "operator": "in",
            "value": filter_dict["material"]
        })
    
    # Handle price filter
    if "price" in filter_dict and isinstance(filter_dict["price"], dict):
        price_filter = filter_dict["price"]
        if "$gte" in price_filter:
            conditions.append({
                "field": "price",
                "operator": ">=",
                "value": price_filter["$gte"]
            })
        if "$lte" in price_filter:
            conditions.append({
                "field": "price", 
                "operator": "<=",
                "value": price_filter["$lte"]
            })
    
    # Return proper format
    if len(conditions) == 0:
        return {}
    elif len(conditions) == 1:
        return conditions[0]
    else:
        return {
            "operator": "AND",
            "conditions": conditions
        }

In [16]:
def retrieve_and_generate(query: Annotated[str, "User query for product recommendations"], 
                         paraphraser, metadata_filter, rag_pipeline):
    """Tool for product recommendations with fixed filter handling"""
    print(f"🔍 Processing product query: '{query}'")
    
    # Step 1: Paraphrase query based on context
    paraphrased_query = paraphraser.run(query)
    print(f"📝 Paraphrased: '{paraphrased_query}'")
    
    # Step 2: Generate metadata filter
    filter_result = metadata_filter.run(paraphrased_query)
    print(f"🎯 Filter result: {filter_result}")
    
    # Extract and normalize filter
    filter_dict = {}
    try:
        # Try to parse as direct JSON first
        filter_dict = json.loads(filter_result.strip())
        print(f"✅ Parsed filter (direct): {filter_dict}")
        
        # Convert to Haystack format if needed
        if not ("field" in filter_dict or "operator" in filter_dict):
            filter_dict = normalize_filter_format(filter_dict)
            print(f"🔄 Normalized filter: {filter_dict}")
            
    except json.JSONDecodeError:
        # Try to extract JSON from markdown
        try:
            json_match = re.search(r'```json\n(.*?)\n```', filter_result, re.DOTALL)
            if json_match:
                json_str = json_match.group(1)
                filter_dict = json.loads(json_str)
                filter_dict = normalize_filter_format(filter_dict)
                print(f"✅ Parsed filter (from markdown): {filter_dict}")
            else:
                print("ℹ️ No filter applied (parsing failed)")
        except Exception as e:
            print(f"⚠️ Filter parsing error: {e}")
            filter_dict = {}
    
    # Step 3: Retrieve and generate recommendations
    result = rag_pipeline.run(paraphrased_query, filter_dict)
    print(f"🛍️ Generated recommendations")
    
    return result

In [17]:
def get_common_information(query: Annotated[str, "User query about shopping information"], 
                          common_info_pipeline):
    """Tool for common shopping information"""
    print(f"ℹ️ Processing info query: '{query}'")
    result = common_info_pipeline.run(query)
    print(f"📋 Generated information response")
    return result


## Test Complete Recommendation System


In [23]:
def main():
    """Main function to test the fixed system"""
    print("Setting up SmartShopper recommendation system...")

    # Document stores
    products_document_store = MongoDBAtlasDocumentStore(
        database_name="smartshopper_store",
        collection_name="products",
        vector_search_index="vector_index",
        full_text_search_index="search_index",
    )

    common_info_document_store = MongoDBAtlasDocumentStore(
        database_name="smartshopper_store",
        collection_name="common_info",
        vector_search_index="common_info_vector_index",
        full_text_search_index="common_info_search_index",
    )
    
    # Chat message store
    chat_message_store = InMemoryChatMessageStore()
    chat_message_writer = ChatMessageWriter(chat_message_store)

    print(f"Products in store: {products_document_store.count_documents()}")
    print(f"Common info in store: {common_info_document_store.count_documents()}")

    # Initialize all components
    print("Initializing all components...")
    paraphraser = ParaphraserPipeline(chat_message_store)
    metadata_filter = MetaDataFilterPipeline(METADATA_FILTER_TEMPLATE)
    rag_pipeline = RetrieveAndGenerateAnswerPipeline(chat_message_store, products_document_store)
    common_info_pipeline = CommonInfoPipeline(common_info_document_store)

    # Create tools
    print("Creating tools...")
    product_tool = Tool(
        name="retrieve_and_generate_recommendation",
        description="Get product recommendations based on user queries about shopping for items.",
        function=partial(retrieve_and_generate, 
                        paraphraser=paraphraser, 
                        metadata_filter=metadata_filter, 
                        rag_pipeline=rag_pipeline),
        parameters={
            "type": "object",
            "properties": {"query": {"type": "string", "description": "User query for products"}},
            "required": ["query"]
        }
    )

    info_tool = Tool(
        name="get_common_information", 
        description="Get information about shipping, payment, returns, and other shopping policies.",
        function=partial(get_common_information, common_info_pipeline=common_info_pipeline),
        parameters={
            "type": "object", 
            "properties": {"query": {"type": "string", "description": "User query about shopping information"}},
            "required": ["query"]
        }
    )

    print("\n=== COMPLETE SYSTEM TESTING ===")

    test_scenarios = [
        # Product queries
        {
            "type": "product",
            "conversation": [
                "Hi, I need a new phone",
                "My budget is around 15 million rupiah", 
                "Show me options with good cameras"
            ]
        },
        # Mixed queries  
        {
            "type": "mixed",
            "conversation": [
                "I want to buy shoes",
                "What's your return policy?",
                "Show me running shoes under 3 million"
            ]
        },
        # Info queries
        {
            "type": "info", 
            "conversation": [
                "How do I make payments?",
                "What about shipping costs?",
                "Can I get customer support?"
            ]
        }
    ]

    for scenario_idx, scenario in enumerate(test_scenarios, 1):
        print(f"\n{'='*60}")
        print(f"SCENARIO {scenario_idx}: {scenario['type'].upper()} QUERIES")
        print('='*60)
        
        # Clear conversation for each scenario
        scenario_chat_store = InMemoryChatMessageStore()
        scenario_chat_writer = ChatMessageWriter(scenario_chat_store)
        scenario_paraphraser = ParaphraserPipeline(scenario_chat_store)
        
        # Update tools with new paraphraser for this scenario
        scenario_product_tool = Tool(
            name="retrieve_and_generate_recommendation",
            description="Get product recommendations based on user queries about shopping for items.",
            function=partial(retrieve_and_generate, 
                            paraphraser=scenario_paraphraser, 
                            metadata_filter=metadata_filter, 
                            rag_pipeline=rag_pipeline),
            parameters={
                "type": "object",
                "properties": {"query": {"type": "string", "description": "User query for products"}},
                "required": ["query"]
            }
        )
        
        for turn_idx, user_query in enumerate(scenario['conversation'], 1):
            print(f"\nTurn {turn_idx}: User: '{user_query}'")
            print("-" * 40)
            
            # Add user message to memory
            scenario_chat_writer.run([ChatMessage.from_user(user_query)])
            
            # Determine which tool to use (simulate agent decision)
            if any(keyword in user_query.lower() for keyword in 
                   ['show', 'need', 'want', 'buy', 'phone', 'shoes', 'options', 'camera']):
                # Use product tool
                print("🤖 Agent Decision: Using product recommendation tool")
                try:
                    result = scenario_product_tool.function(user_query)
                    print("\nAssistant Response:")
                    print(result[:300] + "..." if len(result) > 300 else result)
                    # Add assistant response to memory
                    scenario_chat_writer.run([ChatMessage.from_assistant(result)])
                except Exception as e:
                    print(f"❌ Error with product tool: {e}")
            else:
                # Use info tool  
                print("🤖 Agent Decision: Using common information tool")
                try:
                    result = info_tool.function(user_query)
                    print("\nAssistant Response:")
                    print(result[:300] + "..." if len(result) > 300 else result)
                    # Add assistant response to memory
                    scenario_chat_writer.run([ChatMessage.from_assistant(result)])
                except Exception as e:
                    print(f"❌ Error with info tool: {e}")

    print("\n\n✅ Shop recommendation testing completed!")
    return True

In [24]:

if __name__ == "__main__":
    main()

Setting up SmartShopper recommendation system...
Products in store: 5
Common info in store: 6
Initializing all components...
Creating tools...

=== COMPLETE SYSTEM TESTING ===

SCENARIO 1: PRODUCT QUERIES

Turn 1: User: 'Hi, I need a new phone'
----------------------------------------
🤖 Agent Decision: Using product recommendation tool
🔍 Processing product query: 'Hi, I need a new phone'
📝 Paraphrased: 'You're still looking for a new phone, is there anything specific you're looking for in your new device?'
🎯 Filter result: {"field": "category", "operator": "in", "value": ["Electronics"]}
✅ Parsed filter (direct): {'field': 'category', 'operator': 'in', 'value': ['Electronics']}


Batches: 100%|██████████| 1/1 [00:00<00:00,  7.11it/s]


🛍️ Generated recommendations

Assistant Response:
It seems like you're in the market for a new phone. To give you the best recommendations, could you please provide more details about what you're looking for in your new device? For example, are you interested in a specific operating system like Android or iOS? Are there any particular features that...

Turn 2: User: 'My budget is around 15 million rupiah'
----------------------------------------
🤖 Agent Decision: Using common information tool
ℹ️ Processing info query: 'My budget is around 15 million rupiah'


Batches: 100%|██████████| 1/1 [00:00<00:00, 13.62it/s]


📋 Generated information response

Assistant Response:
Hello! I'd be happy to help you with your query.

Since your budget is around 15 million rupiah, you have a wide range of options to choose from on our website. With this budget, you can definitely take advantage of our free shipping offer, which applies to orders above 500,000 rupiah. This means yo...

Turn 3: User: 'Show me options with good cameras'
----------------------------------------
🤖 Agent Decision: Using product recommendation tool
🔍 Processing product query: 'Show me options with good cameras'
📝 Paraphrased: 'You're looking for phone options within your budget of 15 million rupiah that have high-quality cameras. I can help you with that. Based on your previous input, I'll provide you with some recommendations that fit your criteria and have good camera capabilities.'
🎯 Filter result: {"operator": "AND", "conditions": [{"field": "category", "operator": "in", "value": ["Electronics"]}, {"field": "price", "operator": "<=",

...

📋 Generated information response

Assistant Response:
I'm happy to help you with your payment options.

We offer a variety of convenient payment methods to make your shopping experience with us as smooth as possible. You can choose from the following options:

1. **Credit Cards**: We accept Visa and MasterCard, so if you have either of these, you're al...

Turn 2: User: 'What about shipping costs?'
----------------------------------------
🤖 Agent Decision: Using common information tool
ℹ️ Processing info query: 'What about shipping costs?'


Batches: 100%|██████████| 1/1 [00:00<00:00, 15.02it/s]


📋 Generated information response

Assistant Response:
I'd be happy to help you with your question about shipping costs.

We're excited to offer free shipping for orders above Rp 500,000, so if your order meets that minimum amount, you won't have to pay a single rupiah for shipping. This applies to both standard and express delivery options.

If your or...

Turn 3: User: 'Can I get customer support?'
----------------------------------------
🤖 Agent Decision: Using common information tool
ℹ️ Processing info query: 'Can I get customer support?'


Batches: 100%|██████████| 1/1 [00:00<00:00,  7.65it/s]


📋 Generated information response

Assistant Response:
You're looking for customer support. Don't worry, we're here to help. Our customer support team is available to assist you from Monday to Friday, 9AM-6PM WIB. You can reach out to us through various channels, including:

* WhatsApp: Just send us a message, and we'll get back to you as soon as possib...


✅ Shop recommendation testing completed!
